# Explore here

In [8]:
import pandas as pd
from utils import db_connect
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [2]:
engine = db_connect()

# Consultar la tabla en la base de datos y crear un DataFrame
total_data = pd.read_sql_table('tabla playstore', engine)

pd.set_option('display.max_columns', None)  # muestra todas las columnas del dataframe
total_data.head()

,package_name,review,polarity
0,com.facebook.katana,privacy at least put some option appear offli...,0
1,com.facebook.katana,"messenger issues ever since the last update, ...",0
2,com.facebook.katana,profile any time my wife or anybody has more ...,0
3,com.facebook.katana,the new features suck for those of us who don...,0
4,com.facebook.katana,forced reload on uploading pic on replying co...,0


In [3]:
total_data.to_csv('/workspaces/Naive-Bayes/data/raw/playstore.csv', index=False)

tenemos solo 3 variables: 2 predictoras y una etiqueta dicotómica. De las dos predictoras, realmente solo nos interesa la parte del comentario, ya que el hecho de clasificar un comentario en positivo o negativo dependerá de su contenido, no de la aplicación de la que se haya escrito. Por lo tanto, la variable package_name habría que eliminarla.

In [4]:
total_data.drop(['package_name'], axis=1, inplace=True)

In [5]:
total_data.shape

(891, 2)

In [6]:
total_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   review    891 non-null    object
 1   polarity  891 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 14.0+ KB


In [9]:
total_data.head()

,review,polarity
0,privacy at least put some option appear offli...,0
1,"messenger issues ever since the last update, ...",0
2,profile any time my wife or anybody has more ...,0
3,the new features suck for those of us who don...,0
4,forced reload on uploading pic on replying co...,0


In [10]:
total_data["review"] = total_data["review"].str.strip().str.lower()
total_data.head()

,review,polarity
0,privacy at least put some option appear offlin...,0
1,"messenger issues ever since the last update, i...",0
2,profile any time my wife or anybody has more t...,0
3,the new features suck for those of us who don'...,0
4,forced reload on uploading pic on replying com...,0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(review, test_size=0.2, random_state=42)

In [ ]:
vectorizer = CountVectorizer(stop_words = "english")   # stop_words = "english" eliminará las palabras comunes del inglés del texto antes de generar la matriz de recuentos
X_train = vectorizer.fit_transform(X_train).toarray()   # toarray() convierte la matriz dispersa resultante en una matriz densa.
X_test = vectorizer.transform(X_test).toarray()

X_train = vectorizer.fit_transform(X_train).toarray(): Convierte los documentos de texto en el conjunto de datos de entrenamiento (X_train) en una matriz de recuentos de tokens. fit_transform() primero ajusta el vectorizador al conjunto de datos de entrenamiento (X_train) y luego lo transforma en una matriz de recuentos.  

X_test = vectorizer.transform(X_test).toarray(): Transforma los documentos de texto en el conjunto de datos de prueba (X_test) en una matriz de recuentos de tokens utilizando el mismo vectorizador que se ajustó al conjunto de datos de entrenamiento. transform() simplemente aplica la transformación aprendida previamente en el conjunto de datos de entrenamiento al conjunto de datos de prueba.

 - Matriz dispersa: Solo se almacenan los valores distintos de cero junto con sus ubicaciones, lo que ahorra memoria. Esto es útil cuando se trabaja con matrices muy grandes donde la mayoría de los elementos son cero. Algunos formatos de matriz dispersa comunes son el formato de lista de listas (LIL), el formato de matriz comprimida en fila (CSR) y el formato de matriz comprimida en columna (CSC).

 - Matriz densa: Todos los elementos de la matriz se almacenan en la memoria, independientemente de si son cero o no. Esto puede llevar a un uso significativo de memoria, especialmente para matrices grandes con muchos ceros. Sin embargo, las operaciones en matrices densas suelen ser más rápidas que en matrices dispersas, especialmente en operaciones de álgebra lineal.

La elección entre usar una matriz dispersa o densa depende del contexto y de las características de los datos:

Para conjuntos de datos grandes con muchos ceros, como matrices de documentos término-documento en procesamiento de lenguaje natural, las matrices dispersas son más eficientes en términos de memoria.
Para operaciones de álgebra lineal y cálculos intensivos, como la multiplicación de matrices, las matrices densas suelen ser más eficientes computacionalmente.
Algunos algoritmos de aprendizaje automático pueden requerir entradas en forma de matrices densas, mientras que otros pueden admitir matrices dispersas. Por lo tanto, 